#  ML Capstone Project - Impact of Covid on Student's Physical well being

In [1]:
# Select K Best 


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
# Feature selection using chi2
def selectKFeature(indep_X,dep_Y,n):
    test=SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(indep_X,dep_Y)
    select_k=fit1.transform(indep_X)
    return select_k
    
#Integrated train test split,standard scaler
def split_scaler(indep_X,dep_Y):
    x_train, x_test, y_train, y_test=train_test_split(indep_X,dep_Y,test_size=0.25,random_state=0)
   # for model improvement removed standard scaler
    # sc=StandardScaler()
   # x_train=sc.fit_transform(x_train)
   # x_test=sc.transform(x_test)
    return x_train, x_test, y_train, y_test

# prediction and evalation metrics
def acc_prediction(grid,x_test):
    grid_Predict=grid.predict(x_test)
    acc_Score=grid.best_score_
   # from sklearn.metrics import confusion_matrix
   # cm=confusion_matrix(y_test,y_pred)
    
   # from sklearn.metrics import classification_report
   # clf_report=classification_report(y_test,y_pred)
    
    #from sklearn.metrics import accuracy_score
   # accuracy=accuracy_score(y_test,y_pred)
    return grid_Predict,acc_Score

def logistic(x_train, y_train, x_test):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV
    param_grid={ 'penalty':[ 'l2',],
            'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
           }
    grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='accuracy')
    grid.fit(x_train,y_train)
    grid_Predict,acc_Score = acc_prediction(grid,x_test)
    return grid_Predict,acc_Score

def svm_linear(x_train, y_train, x_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV
    param_grid={ 'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma' :['scale', 'auto'] 
           }
    grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='accuracy')
    grid.fit(x_train,y_train)
    grid_Predict,acc_Score = acc_prediction(grid,x_test)
    return grid_Predict,acc_Score

def naivebayes(x_train, y_train, x_test):
    from sklearn.naive_bayes import BernoulliNB
    classifier=BernoulliNB()
    classifier.fit(x_train, y_train)
    
    y_pred=classifier.predict(x_test)
    
    from sklearn.metrics import accuracy_score
    accuracy=accuracy_score(y_test,y_pred)
    return y_pred,accuracy

def knn(x_train, y_train, x_test):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    param_grid={ 'n_neighbors' :[4,5] ,'weights' :['uniform', 'distance'] ,'metric':['minkowski'],'p':[4,5]
           }
    #best model must be selected on the basis of scoring f1_weighted
    grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='accuracy')
    grid.fit(x_train,y_train)
    grid_Predict,acc_Score = acc_prediction(grid,x_test)
    return grid_Predict,acc_Score
    
   
def randomForest(x_train, y_train, x_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    param_grid={ 'n_estimators' : [25,50,75], 'criterion' : ['gini', 'entropy', 'log_loss'] , 
                 'max_features' : ['sqrt', 'log2', None]
           }
    #best model must be selected on the basis of scoring accuracy
    grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='accuracy')
    grid.fit(x_train,y_train)
    grid_Predict,acc_Score = acc_prediction(grid,x_test)
    return grid_Predict,acc_Score

def decisionTree(x_train, y_train, x_test):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import GridSearchCV
    param_grid={ 'criterion' : ['gini', 'entropy', 'log_loss'] ,
                 'splitter' : ['best', 'random'] , 'max_features' : ['sqrt', 'log2']
           }
    #best model must be selected on the basis of scoring accuracy
    grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='accuracy')
    grid.fit(x_train,y_train)
    grid_Predict,acc_Score = acc_prediction(grid,x_test)
    return grid_Predict,acc_Score

def selectK_table(acclog,accknn,accnaive,accdes,accrf):
    dataframe=pd.DataFrame(index=['Chi Square'],columns=['logistic','Naive','KNN','DTree','RForest'])
    for number,idex in enumerate(dataframe.index):
        dataframe['logistic'][idex]=acclog[number]
       # dataframe['svm'][idex]=accsvm_l[number]
        dataframe['Naive'][idex]=accnaive[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['DTree'][idex]=accdes[number]
        dataframe['RForest'][idex]=accrf[number]
        return dataframe

# Data Collection

In [3]:
# Data Collection
dataset=pd.read_csv('preCovidStudentSurvey.csv',index_col=None)

In [4]:
df=dataset

In [5]:
df

,Age of Subject,Time spent on Online Class,Rating of Online Class experience,Medium for online class,Time spent on self study,Time spent on fitness,Time spent on sleep,Time spent on social media,Prefered social media platform,Time spent on TV,Number of meals per day,Change in your weight,Health issue during lockdown,Stress busters,Time utilized,"Do you find yourself more connected with your family, close friends , relatives ?",What you miss the most
0,21,2.0,Good,Laptop/Desktop,4.0,0.0,7.0,3.0,Linkedin,1.0,4.0,Increased,NO,Cooking,YES,YES,School/college
1,21,0.0,Excellent,Smartphone,0.0,2.0,10.0,3.0,Youtube,0.0,3.0,Decreased,NO,Scrolling through social media,YES,NO,Roaming around freely
2,20,7.0,Very poor,Laptop/Desktop,3.0,0.0,6.0,2.0,Linkedin,0.0,3.0,Remain Constant,NO,Listening to music,NO,YES,Travelling
3,20,3.0,Very poor,Smartphone,2.0,1.0,6.0,5.0,Instagram,0.0,3.0,Decreased,NO,Watching web series,NO,NO,"Friends , relatives"
4,21,3.0,Good,Laptop/Desktop,3.0,1.0,8.0,3.0,Instagram,1.0,4.0,Remain Constant,NO,Social Media,NO,NO,Travelling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,12,3.0,Good,Smartphone,4.0,1.0,8.0,1.0,Instagram,2.0,3.0,Decreased,NO,Dancing,YES,YES,Travelling
1178,14,6.0,Average,Smartphone,4.0,1.0,9.0,1.0,WhatsApp,1.0,4.0,Remain Constant,NO,Listening to music,YES,YES,"Friends , relatives"
1179,13,4.0,Average,Smartphone,0.0,0.5,8.0,3.0,Youtube,2.0,4.0,Decreased,NO,Online gaming,NO,YES,School/college
1180,14,5.0,Excellent,Laptop/Desktop,3.5,1.0,8.0,0.5,Youtube,1.0,4.0,Remain Constant,NO,Reading books,YES,YES,School/college


# Data Preprocessing

In [6]:
# Data Preprocessing
df=pd.get_dummies(df,drop_first=True)


In [7]:
df

,Age of Subject,Time spent on Online Class,Time spent on self study,Time spent on fitness,Time spent on sleep,Time spent on social media,Time spent on TV,Number of meals per day,Rating of Online Class experience_Excellent,Rating of Online Class experience_Good,...,What you miss the most_Playing,What you miss the most_Previous mistakes,What you miss the most_Roaming around freely,What you miss the most_School and friends.,What you miss the most_School/college,What you miss the most_The idea of being around fun loving people but this time has certainly made us all to reconnect (and fill the gap if any) with our families and relatives so it is fun but certainly we do miss hanging out with friends,What you miss the most_To stay alone.,What you miss the most_Travelling,What you miss the most_all of the above,What you miss the most_everything
0,21,2.0,4.0,0.0,7.0,3.0,1.0,4.0,False,True,...,False,False,False,False,True,False,False,False,False,False
1,21,0.0,0.0,2.0,10.0,3.0,0.0,3.0,True,False,...,False,False,True,False,False,False,False,False,False,False
2,20,7.0,3.0,0.0,6.0,2.0,0.0,3.0,False,False,...,False,False,False,False,False,False,False,True,False,False
3,20,3.0,2.0,1.0,6.0,5.0,0.0,3.0,False,False,...,False,False,False,False,False,False,False,False,False,False
4,21,3.0,3.0,1.0,8.0,3.0,1.0,4.0,False,True,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,12,3.0,4.0,1.0,8.0,1.0,2.0,3.0,False,True,...,False,False,False,False,False,False,False,True,False,False
1178,14,6.0,4.0,1.0,9.0,1.0,1.0,4.0,False,False,...,False,False,False,False,False,False,False,False,False,False
1179,13,4.0,0.0,0.5,8.0,3.0,2.0,4.0,False,False,...,False,False,False,False,True,False,False,False,False,False
1180,14,5.0,3.5,1.0,8.0,0.5,1.0,4.0,True,False,...,False,False,False,False,True,False,False,False,False,False


In [8]:
df.columns

Index(['Age of Subject', 'Time spent on Online Class',
       'Time spent on self study', 'Time spent on fitness',
       'Time spent on sleep', 'Time spent on social media', 'Time spent on TV',
       'Number of meals per day',
       'Rating of Online Class experience_Excellent',
       'Rating of Online Class experience_Good',
       ...
       'What you miss the most_Playing',
       'What you miss the most_Previous mistakes',
       'What you miss the most_Roaming around freely',
       'What you miss the most_School and friends.',
       'What you miss the most_School/college',
       'What you miss the most_The idea of being around fun loving people but this time has certainly made us all to reconnect (and fill the gap if any) with our families and relatives so it is fun but certainly we do miss hanging out with friends',
       'What you miss the most_To stay alone. ',
       'What you miss the most_Travelling',
       'What you miss the most_all of the above',
       'What you

# input output split

In [9]:
# input output split
indep_x=df.drop('Health issue during lockdown_YES',axis=1)
dep_y=df['Health issue during lockdown_YES']

# Feature Selection Using chi 2

In [18]:
# Feature Selection Using chi 2
KBest=selectKFeature(indep_x,dep_y,10)

acclog=[]
accsvm_l=[]

accknn=[]
accnaive=[]
accdes=[]
accrf=[]

In [19]:
KBest

array([[21.,  2.,  0., ...,  0.,  0.,  0.],
       [21.,  0.,  0., ...,  0.,  0.,  0.],
       [20.,  7.,  1., ...,  0.,  0.,  0.],
       ...,
       [13.,  4.,  0., ...,  0.,  0.,  0.],
       [14.,  5.,  0., ...,  0.,  0.,  0.],
       [13.,  5.,  0., ...,  0.,  0.,  0.]])

# Train and test set split

In [20]:
x_train,x_test,y_train,y_test=split_scaler(KBest,dep_y)

# Our output column in in the form of YES or NO so we chose classification algorithms

In [21]:
grid_Predict,acc_Score=logistic(x_train,y_train,x_test)
acclog.append(acc_Score)

#SVM algo takes more time to execute so svm_linear() is didn't called....
#grid_Predict,acc_Score=svm_linear(x_train,y_train,x_test)
#accsvm_l.append(acc_Score)

grid_Predict,acc_Score=naivebayes(x_train,y_train,x_test)
accnaive.append(acc_Score)

grid_Predict,acc_Score=knn(x_train,y_train,x_test)
accknn.append(acc_Score)

grid_Predict,acc_Score=randomForest(x_train,y_train,x_test)
accrf.append(acc_Score)

grid_Predict,acc_Score=decisionTree(x_train,y_train,x_test)
accdes.append(acc_Score)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [22]:
result=selectK_table(acclog,accknn,accnaive,accdes,accrf)

In [40]:
result
#7

,logistic,svm,Naive,KNN,DTree,RForest
Chi Square,0.857792,0.860052,0.878378,0.844252,0.826173,0.826179


In [46]:
result
#8

,logistic,svm,Naive,KNN,DTree,RForest
Chi Square,0.858922,0.861182,0.878378,0.844252,0.828426,0.829563


In [17]:
result
#9

,logistic,Naive,KNN,DTree,RForest
Chi Square,0.860046,0.881757,0.840849,0.827309,0.830693


In [23]:
result
#10

,logistic,Naive,KNN,DTree,RForest
Chi Square,0.860046,0.881757,0.840849,0.825043,0.826173


# Naive Bayes got highest accuracy score . Finding which feature plays the vital role for naive algorithm

In [69]:
test=SelectKBest(score_func=chi2,k=10)
fit1=test.fit(indep_x,dep_y)
select_k=fit1.transform(indep_x)

In [70]:
x_train, x_test, y_train, y_test=train_test_split(select_k,dep_y,test_size=0.25,random_state=0)

In [71]:
from sklearn.naive_bayes import BernoulliNB
classifier=BernoulliNB()
classifier.fit(x_train, y_train)


BernoulliNB()

In [72]:
selected_features = test.get_support(indices=True)
selected_features

array([  0,   1,  12,  21,  27, 105, 106, 111, 118, 129], dtype=int64)

In [73]:
selected_feature_names = df.columns[selected_features]

In [74]:
print("Selected Feature Names:", selected_feature_names)

Selected Feature Names: Index(['Age of Subject', 'Time spent on Online Class',
       'Rating of Online Class experience_Very poor',
       'Prefered social media platform_Others',
       'Change in your weight_Remain Constant',
       'Stress busters_listening to music,reading books and dancing.',
       'Stress busters_live stream watching',
       'Stress busters_watching movies,reading books,games,listening to music,sleep,dancing',
       'What you miss the most_All of the above ',
       'What you miss the most_Gym'],
      dtype='object')


In [75]:
y_pred=classifier.predict(x_test)

In [76]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [77]:
accuracy

0.8817567567567568

In [78]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(x_test)[:,1])

0.6034644683293333

In [79]:
import pickle
filename="GoodModel_Naive.sav"
pickle.dump(classifier,open(filename,'wb'))

'Age of Subject', 'Time spent on Online Class',
       'Rating of Online Class experience_Very poor',
       'Prefered social media platform_Others',
       'Change in your weight_Remain Constant', 'Stress busters_Youtube',
       'Stress busters_listening to music,reading books and dancing.',
       'Stress busters_live stream watching',
       'Stress busters_watching movies,reading books,games,listening to music,sleep,dancing',
       'What you miss the most_All of the above ',
       'What you miss the most_Gym'

# Checking using predict() before deployement phase

In [81]:
loaded_model=pickle.load(open("GoodModel_Naive.sav",'rb'))
result=loaded_model.predict([[21,5,True,True,False,True,True,False,True,False]])

In [82]:
print("Future Predictions where the HEALTH ISSUE arises for selected feature: {} ".format(result))

Future Predictions where the HEALTH ISSUE arises for selected feature: [ True] 
